# Web Scraping: Which sellers advertise/sponsor 'Ipad4' on Ebay

The following notebook is an end to end process of scraping product (Ipad4) information from Ebay and saving it in a MySQL table to conduct data wranglings and manipulations. It has the following steps:

1)Storing sponsored item and non-sponsored item url links for Ipad4 for the first 10 pages on Ebay as 2 separate txt files

2)For both sponsored and non-sponsored items, downloading every item page as a htm file in respective folders

3)Scraping different metrics for each item for both sponsored and non-sponsored types

4)Creating MySQL database and tables for sponsored and non-sponsored items

5)Analysing this data to identify the types of sellers who advertise/sponsor 'Ipad4' on Ebay

In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
from time import sleep
import os
import re
from tqdm import tqdm
import pandas as pd
import pymysql

### Storing sponsored and non-sponsored item url links for the first 10 pages in sponsored.txt and non-sponsored.txt

In [5]:
### Define variables and lists 
pages = [str(i) for i in range(1,11)]
sponsored_links = []
nonsponsored_links=[]
link_regex = re.compile(r"(.*)\?.*")

###Run loop for 10 pages 
for page in pages:
    
    ### Get the page details in soup element
    
    url_page="https://www.ebay.com/sch/i.html?_from=R40&_nkw=ipad+4&_sacat=0&rt=nc&LH_BIN=1&_ipg=100&_pgn="+page
    response_page= requests.get(url_page,agent)
    page_soup=bs(response_page.text,"html.parser")
    page_soup
    h3 = page_soup.findAll("h3")
    for element in h3:
        
        ###identify if item is sponsored  and store item link in sponsored.txt
        
        if("s-item__title--has-tags" in str(element["class"])): # is a sponsored
            s_link = link_regex.sub(r'\1',element.parent['href'])
            sponsored_links.append(s_link)
            with open("sponsored.txt", "a") as file:
                file.write(s_link)
                file.write("\n")
                
        ###identify if item is non-sponsored and store item link in non-sponsored.txt
        
        elif("s-item__title" in str(element["class"])):
            ns_link = link_regex.sub(r'\1',element.parent['href'])
            nonsponsored_links.append(ns_link)
            with open("non_sponsored.txt", "a") as file:
                file.write(ns_link)
                file.write("\n")
    sleep(2)
    print("Page "+page+" completed.")    

Page 1 completed.
Page 2 completed.
Page 3 completed.
Page 4 completed.
Page 5 completed.
Page 6 completed.
Page 7 completed.
Page 8 completed.
Page 9 completed.
Page 10 completed.


In [6]:
###Get current working directory
current_working_directory=os.getcwd()
current_working_directory

'C:\\Users\\Shrey Shah\\Documents\\GitHub\\DDR'

In [7]:
###Make 2 new folders as 'sponsored' and "non-sponsored"
os.mkdir("sponsored")
os.mkdir("non-sponsored")

In [8]:
###Declare variables for sponsored and non-sponsored working directories
directory_sponsored = current_working_directory+"/sponsored"
directory_non_sponsored = current_working_directory+"/non-sponsored"
print(directory_sponsored)
print(directory_non_sponsored)

C:\Users\Shrey Shah\Documents\GitHub\DDR/sponsored
C:\Users\Shrey Shah\Documents\GitHub\DDR/non-sponsored


In [9]:
os.chdir(current_working_directory)

### Read all the item url links from sponsored.txt
with open("sponsored.txt","r") as file:
    all_lines = file.readlines()

In [10]:
### Change working directory to sponsored folder

os.chdir(directory_sponsored)
os.getcwd()

'C:\\Users\\Shrey Shah\\Documents\\GitHub\\DDR\\sponsored'

### For sponsored items,download every item page and store as htm file in sponsored folder

In [11]:
### For sponsored items,download every item page and store as htm file in sponsored folder

sponsored_items = []
for i in tqdm(range(0,len(all_lines))):
    url_item_line=all_lines[i]
    url_item_line = url_item_line.replace("\n","")
    try:
        response_item = requests.get(url_item_line,agent)
        sleep(2)
        item_regex = re.compile('.*(itm/[a-zA-Z0-9-]+/)(.*)')
        current_itemid = item_regex.sub(r'\2',url_item_line)
    except Exception:
        current_itemid = i
        print("Error occured at",i)
    sponsored_items.append(current_itemid)
    with open(current_itemid + ".htm", "w", encoding = 'utf-8') as file:
        file.write(response_item.text)

100%|████████████████████████████████████████████████████████████████████████████████| 130/130 [06:00<00:00,  2.90s/it]


In [12]:
os.chdir(current_working_directory)

### Read all the item url links from non_sponsored.txt
with open("non_sponsored.txt","r") as file:
    all_lines = file.readlines()

In [13]:
### Change working directory to non-sponsored folder

os.chdir(directory_non_sponsored)
os.getcwd()

'C:\\Users\\Shrey Shah\\Documents\\GitHub\\DDR\\non-sponsored'

### For non-sponsored items,download every item page and store as htm file in non-sponsored folder

In [14]:
### For non-sponsored items,download every item page and store as htm file in non-sponsored folder

non_sponsored_items = []
for i in tqdm(range(0,len(all_lines))):
    url_item_line=all_lines[i]
    url_item_line = url_item_line.replace("\n","")
    try:
        response_item = requests.get(url_item_line,agent)
        item_regex = re.compile('.*(itm/[a-zA-Z0-9-]+/)(.*)')
        current_itemid = item_regex.sub(r'\2',url_item_line)
    except Exception as e:
        print("Error occured at",i,'\n' ,e)
        current_itemid = str(i)
    non_sponsored_items.append(current_itemid)
    with open(current_itemid + ".htm", "w", encoding = 'utf-8') as file:
        file.write(response_item.text)
        file.close()
    sleep(2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [45:47<00:00,  2.74s/it]


In [15]:
### Changing working directory to sponsored folder
os.chdir(directory_sponsored)
os.getcwd()

'C:\\Users\\Shrey Shah\\Documents\\GitHub\\DDR\\sponsored'

### Getting Metrics for Sponsored Items 

In [16]:
###Declaring storing variable lists for sponsored items 

soup_pages_s = []
sellernames_s=[]
sellerscores_s=[]
itemprices_s=[]
itemssold_s=[]
bestoffers_s=[]
titles_s=[]
returns_allowed_s=[]
shipping_prices_s=[]
conditions_s=[]


### Running loop for every sponsored item to get different metrics and informations
for i in range(0,len(sponsored_items)):
    with open(sponsored_items[i]+".htm",encoding ='utf-8') as file:
        soup = bs(file.read(),"html.parser")
        soup_pages_s.append(soup)
        file.close()

### Getting sellername
    sellername = soup.find("span",class_="mbg-nw").text
    sellernames_s.append(sellername)

### Getting seller score 
    sellerscore = soup.find("span",class_="mbg-l").find("a").text
    sellerscores_s.append(sellerscore)

### Getting item price    
    itemprice = soup.find("span",class_="notranslate").text
    itemprices_s.append(itemprice)

### Getting number of items sold 
    itemsold = soup.find("a",class_="vi-txt-underline")
    if (itemsold!=None):
        itemsold=itemsold.text
    else:
        itemsold=None
    itemssold_s.append(itemsold)

### Getting if item has best offer
    bestoffer = soup.find("div", class_="vi-bbox-dspn u-flL lable boLable")
    if (bestoffer!=None):
        bestoffer=bestoffer.text
    else:
        bestoffer=None
    bestoffers_s.append(bestoffer)

### Getting title for item 
    title = soup.find("h1", class_="it-ttl")
    title = title.text
    titles_s.append(title)

### Getting information if returns allowed for item
    return_allowed = soup.find("span",id="vi-ret-accrd-txt")
    if (return_allowed!=None):
        return_allowed=return_allowed.text
    else:
        return_allowed=None
    returns_allowed_s.append(return_allowed) 

### Getting shipping price for item   
    shipping_price = soup.find("span", id="shSummary").find("span").text
    shipping_prices_s.append(shipping_price)

### Getting current condition of the item
    condition = soup.find("div", id="vi-itm-cond").text
    conditions_s.append(condition)

### Check progress    
    print("Metrics for Item",i+1,"have been computed")
    

    

Metrics for Item 1 have been computed
Metrics for Item 2 have been computed
Metrics for Item 3 have been computed
Metrics for Item 4 have been computed
Metrics for Item 5 have been computed
Metrics for Item 6 have been computed
Metrics for Item 7 have been computed
Metrics for Item 8 have been computed
Metrics for Item 9 have been computed
Metrics for Item 10 have been computed
Metrics for Item 11 have been computed
Metrics for Item 12 have been computed
Metrics for Item 13 have been computed
Metrics for Item 14 have been computed
Metrics for Item 15 have been computed
Metrics for Item 16 have been computed
Metrics for Item 17 have been computed
Metrics for Item 18 have been computed
Metrics for Item 19 have been computed
Metrics for Item 20 have been computed
Metrics for Item 21 have been computed
Metrics for Item 22 have been computed
Metrics for Item 23 have been computed
Metrics for Item 24 have been computed
Metrics for Item 25 have been computed
Metrics for Item 26 have been comp

### Getting metrics for Non-Sponsored Items 

In [17]:
### Change working directory to non-sponsored folder
os.chdir(directory_non_sponsored)
os.getcwd()

'C:\\Users\\Shrey Shah\\Documents\\GitHub\\DDR\\non-sponsored'

In [18]:
### Declaring storing variable lists for sponsored items 

soup_pages_ns = []
sellernames_ns=[]
sellerscores_ns=[]
itemprices_ns=[]
itemssold_ns=[]
bestoffers_ns=[]
titles_ns=[]
returns_allowed_ns=[]
shipping_prices_ns=[]
conditions_ns=[]

### Running loop for every non-sponsored item to get different metrics and informations
for i in range(0,len(non_sponsored_items)):
    with open(non_sponsored_items[i]+".htm",encoding ='utf-8') as file:
        soup = bs(file.read(),"html.parser")
        soup_pages_ns.append(soup)
        file.close()
        
### Getting sellername
    sellername = soup.find("span",class_="mbg-nw").text
    sellernames_ns.append(sellername)

### Getting seller score
    sellerscore = soup.find("span",class_="mbg-l").find("a").text
    sellerscores_ns.append(sellerscore)

### Getting item price 
    itemprice = soup.find("span",class_="notranslate").text
    itemprices_ns.append(itemprice)

### Getting number of items sold 
    itemsold = soup.find("a",class_="vi-txt-underline")
    if (itemsold!=None):
        itemsold=itemsold.text
    else:
        itemsold=None
    itemssold_ns.append(itemsold)

### Getting if item has best offer
    bestoffer = soup.find("div", class_="vi-bbox-dspn u-flL lable boLable")
    if (bestoffer!=None):
        bestoffer=bestoffer.text
    else:
        bestoffer=None
    bestoffers_ns.append(bestoffer)

### Getting title for item
    title = soup.find("h1", class_="it-ttl")
    title = title.text
    titles_ns.append(title)

### Getting information if returns allowed for item
    return_allowed = soup.find("span",id="vi-ret-accrd-txt")
    if (return_allowed!=None):
        return_allowed=return_allowed.text
    else:
        return_allowed=None
    returns_allowed_ns.append(return_allowed) 

### Getting shipping price for item  
    shipping_price = soup.find("span", id="shSummary").find("span").text
    shipping_prices_ns.append(shipping_price)

### Getting current condition of the item
    condition = soup.find("div", id="vi-itm-cond").text
    conditions_ns.append(condition)

### Check progress     
    print("Metrics for Item",i+1,"have been computed")
    

    

Metrics for Item 1 have been computed
Metrics for Item 2 have been computed
Metrics for Item 3 have been computed
Metrics for Item 4 have been computed
Metrics for Item 5 have been computed
Metrics for Item 6 have been computed
Metrics for Item 7 have been computed
Metrics for Item 8 have been computed
Metrics for Item 9 have been computed
Metrics for Item 10 have been computed
Metrics for Item 11 have been computed
Metrics for Item 12 have been computed
Metrics for Item 13 have been computed
Metrics for Item 14 have been computed
Metrics for Item 15 have been computed
Metrics for Item 16 have been computed
Metrics for Item 17 have been computed
Metrics for Item 18 have been computed
Metrics for Item 19 have been computed
Metrics for Item 20 have been computed
Metrics for Item 21 have been computed
Metrics for Item 22 have been computed
Metrics for Item 23 have been computed
Metrics for Item 24 have been computed
Metrics for Item 25 have been computed
Metrics for Item 26 have been comp

### Data Transformations 

In [19]:
### Transforming itemprice column into dollar-cent format for sponsored items 
itemprices_s_modified = []
for i in range(0,len(itemprices_s)):
    itemprice_new = re.findall(r'(\d+[.,]\d+)',itemprices_s[i])[0]
    itemprice_new = itemprice_new.replace(",","")
    itemprice_new = int(float(itemprice_new)*100)
    itemprices_s_modified.append(itemprice_new)

### Transforming itemprice column into dollar-cent format for non-sponsored items 
itemprices_ns_modified = []
for i in range(0,len(itemprices_ns)):
    try:
        itemprice_new = re.findall(r'(\d+[.,]\d+)',itemprices_ns[i])[0]
        itemprice_new = itemprice_new.replace(",","")
        itemprice_new = int(float(itemprice_new)*100)
    except IndexError:
        itemprice_new = None
    itemprices_ns_modified.append(itemprice_new)

In [20]:
### Transforming items-sold into digits format for sponsored items 
itemsold_re = re.compile(r'([0-9,]+).*')
itemssold_s_modified = []
for i in range(0,len(itemssold_s)):
    if (itemssold_s[i]!=None):
        itemsold_new = itemsold_re.sub(r'\1',itemssold_s[i])
        itemsold_new = itemsold_new.replace(",","")
    else:
        itemsold_new = None
    itemssold_s_modified.append(itemsold_new)

### Transforming items-sold into digits format for non-sponsored items 
itemssold_ns_modified = []
for i in range(0,len(itemssold_ns)):
    if (itemssold_ns[i]!=None):
        itemsold_new = itemsold_re.sub(r'\1',itemssold_ns[i])
        itemsold_new = itemsold_new.replace(",","")
    else:
        itemsold_new = None
    itemssold_ns_modified.append(itemsold_new)
   

In [21]:
### Transforming bestoffer into 1 or 0 for sponsored items 
bestoffers_s_modified = []
for i in range(0,len(bestoffers_s)):
    if bestoffers_s[i]!=None:
        bestoffer_new = 1
    else:
        bestoffer_new = 0 
    bestoffers_s_modified.append(bestoffer_new)

### Transforming bestoffer into 1 or 0 for non-sponsored items 
bestoffers_ns_modified = []
for i in range(0,len(bestoffers_ns)):
    if bestoffers_ns[i]!=None:
        bestoffer_new = 1
    else:
        bestoffer_new = 0 
    bestoffers_ns_modified.append(bestoffer_new)


In [22]:
### Cleaning title for sponsored items 
title_re = re.compile(r'^(Details about)(\s+)(\w.*)')
titles_s_modified = []
for i in range(0,len(titles_s)):
    title_new = title_re.sub(r'\3',titles_s[i])
    titles_s_modified.append(title_new)

### Cleaning title for non-sponsored items 
titles_ns_modified = []
for i in range(0,len(titles_ns)):
    title_new = title_re.sub(r'\3',titles_ns[i])
    titles_ns_modified.append(title_new)
   

In [23]:
### Transforming returns allowed into 1 or 0 for sponsored items 
returns_allowed_s_modified = []
for i in range(0,len(returns_allowed_s)):
    if('Seller does not accept returns' in str(returns_allowed_s[i])):
        returns_allowed_new = 0
    elif returns_allowed_s[i]==None:
        returns_allowed_new = 0
    else:
        returns_allowed_new = 1 
    returns_allowed_s_modified.append(returns_allowed_new)

### Transforming returns allowed into 1 or 0 for non-sponsored items    
returns_allowed_ns_modified = []
for i in range(0,len(returns_allowed_ns)):
    if('Seller does not accept returns' in str(returns_allowed_ns[i])):
        returns_allowed_new = 0
    elif returns_allowed_ns[i]==None:
        returns_allowed_new = 0
    else:
        returns_allowed_new = 1 
    returns_allowed_ns_modified.append(returns_allowed_new)
   

In [24]:
### Transforming shipping price into dollar-cent format when detected, 0 if Free, Null when not detected for sponsored items 
ship_price_re = re.compile(r'(\d+.\d+)')
shipping_prices_s_modified = []
for i in range(0,len(shipping_prices_s)):
    if ("$" in str(shipping_prices_s[i])):
        shippingprice_new = re.findall(r'(\d+.\d+)',shipping_prices_s[i])[0]
        shippingprice_new = int(float(shippingprice_new)*100)
    elif (('FAST' in str(shipping_prices_s[i])) or ('FREE' == str(shipping_prices_s[i]))):
        shippingprice_new = 0
    else:
        shippingprice_new = None
    shipping_prices_s_modified.append(shippingprice_new)

### Transforming shipping price into dollar-cent format when detected, 0 if Free, Null when not detected for non-sponsored items    
shipping_prices_ns_modified = []
for i in range(0,len(shipping_prices_ns)):
    if ("$" in str(shipping_prices_ns[i])):
        shippingprice_new = re.findall(r'(\d+.\d+)',shipping_prices_ns[i])[0]
        shippingprice_new = int(float(shippingprice_new)*100)
    elif (('FAST' in str(shipping_prices_ns[i])) or ('FREE' == str(shipping_prices_ns[i]))):
        shippingprice_new = 0
    else:
        shippingprice_new = None
    shipping_prices_ns_modified.append(shippingprice_new)


### Creating eBay Database and eBay_items table in MYSQL 

In [25]:
password_sql = ''
conn = pymysql.connect(host='localhost', user = 'root', password = password_sql)
cursor = conn.cursor()
DB = "eBay"
cursor.execute('DROP DATABASE IF EXISTS '+DB)
query_DB = "CREATE DATABASE IF NOT EXISTS " + DB + ";"
cursor.execute(query_DB)
Table = "eBay_items"
cursor.execute('DROP TABLE IF EXISTS ' + DB + '.' + Table)
query_Table = "CREATE TABLE IF NOT EXISTS " + DB + "." + Table + "(" + "itemid VARCHAR(13) NOT NULL PRIMARY KEY" + ", sellername VARCHAR(50)" + ", sellerscore int(7)" + ", itemprice int(6)" + ", itemsold int(4)" + ", bestoffer_flag boolean" + ", title VARCHAR(500)" + ", returnsallowed_flag boolean" + ", shippingprice int(10)" + ", conditions VARCHAR(50)" + ", sponsor_flag boolean" + ");"

cursor.execute(query_Table);
cursor.close()
conn.close()

C:\Users\Shrey Shah\AppData\Local\Continuum\anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1008, "Can't drop database 'ebay'; database doesn't exist")
  result = self._query(query)
C:\Users\Shrey Shah\AppData\Local\Continuum\anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1051, "Unknown table 'ebay.ebay_items'")
  result = self._query(query)
C:\Users\Shrey Shah\AppData\Local\Continuum\anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1681, 'Integer display width is deprecated and will be removed in a future release.')
  result = self._query(query)


### Inserting data for Sponsored Items into Above Created Table 

In [26]:
### Setup Connection with MySQL
conn = pymysql.connect(host='localhost', 
                       user = 'root', 
                       password = password_sql, 
                       database=DB)
cursor = conn.cursor()

### Run loop for inserting values into table
for i in tqdm(range(0,len(sponsored_items))):
    sql = "INSERT INTO "+ Table + ("(itemid,sellername,sellerscore,itemprice,itemsold,bestoffer_flag,title,returnsallowed_flag,shippingprice,conditions,sponsor_flag)"
        " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 1)")
    values = (sponsored_items[i], 
              sellernames_s[i], 
              sellerscores_s[i], 
              itemprices_s_modified[i], 
              itemssold_s_modified[i],
              bestoffers_s_modified[i], 
              titles_s_modified[i], 
              returns_allowed_s_modified[i], 
              shipping_prices_s_modified[i],
              conditions_s[i])
    
    # Calling the query to be executed 
    try:
        cursor.execute(sql, values)
        conn.commit()
    except pymysql.IntegrityError:
        print(str(sponsored_items[i]) + ' already exists in the database.')

 30%|████████████████████████                                                        | 39/130 [00:00<00:00, 148.98it/s]

181710891072 already exists in the database.


 78%|█████████████████████████████████████████████████████████████▉                 | 102/130 [00:00<00:00, 162.95it/s]

262604947247 already exists in the database.


100%|███████████████████████████████████████████████████████████████████████████████| 130/130 [00:00<00:00, 189.11it/s]


### Inserting data for Non-Sponsored Items into Above Created Table 

In [27]:
# Setup connection with MySQL
conn = pymysql.connect(host='localhost', 
                       user = 'root', 
                       password = password_sql, 
                       database=DB)
cursor = conn.cursor()

### Run loop for inserting values into table
for i in tqdm(range(0,len(non_sponsored_items))):
    sql = "INSERT INTO "+ Table + ("(itemid,sellername,sellerscore,itemprice,itemsold,bestoffer_flag,title,returnsallowed_flag,shippingprice,conditions,sponsor_flag)"
        " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 0)")
    values = (non_sponsored_items[i], 
              sellernames_ns[i], 
              sellerscores_ns[i], 
              itemprices_ns_modified[i], 
              itemssold_ns_modified[i],
              bestoffers_ns_modified[i], 
              titles_ns_modified[i], 
              returns_allowed_ns_modified[i], 
              shipping_prices_ns_modified[i],
              conditions_ns[i])
    
    # Calling the query to be executed 
    try:
        cursor.execute(sql, values)
        conn.commit()
    except pymysql.IntegrityError:
        print(str(non_sponsored_items[i]) + ' already exists in the database.')

  8%|██████▍                                                                        | 82/1000 [00:00<00:05, 160.90it/s]

372729878272 already exists in the database.


 19%|██████████████▌                                                               | 187/1000 [00:01<00:04, 170.00it/s]

181809799186 already exists in the database.


 26%|████████████████████                                                          | 258/1000 [00:01<00:04, 166.00it/s]

...

 79%|█████████████████████████████████████████████████████████████▌                | 790/1000 [00:04<00:01, 177.68it/s]

352648427375 already exists in the database.


 90%|█████████████████████████████████████████████████████████████████████▉        | 896/1000 [00:05<00:00, 154.89it/s]

114010719079 already exists in the database.
192832373115 already exists in the database.
303474693487 already exists in the database.
143338521826 already exists in the database.
113950277133 already exists in the database.


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 167.45it/s]


### Getting the SQL table into Python Dataframe 

In [28]:
conn = pymysql.connect(host='localhost', user = 'root', password = password_sql, database=DB)

Query = pd.read_sql_query('''select itemid,sponsor_flag,sellerscore,itemprice,itemsold,bestoffer_flag,returnsallowed_flag,shippingprice,conditions
from eBay_items''', conn)

eBay_items_df = pd.DataFrame(Query, columns=['itemid','sponsor_flag','sellerscore','itemprice','itemsold','bestoffer_flag','returnsallowed_flag','shippingprice','conditions'])

conn.close()

eBay_items_df.head()

,itemid,sponsor_flag,sellerscore,itemprice,itemsold,bestoffer_flag,returnsallowed_flag,shippingprice,conditions
0,111121377380,0,1739,84889.0,6.0,0,1,NaN,New
1,111320062974,0,1739,79889.0,2.0,0,1,NaN,New
2,111352455994,0,1739,82889.0,NaN,0,1,NaN,New
3,111378999421,0,1739,89899.0,NaN,0,1,NaN,New
4,111524384363,0,1739,99989.0,1.0,0,1,2498.0,New


### Getting min, max and mean for all numerical/binary variables grouped by sponsor_flag and conditions

In [29]:
conn = pymysql.connect(host='localhost', user = 'root', password = password_sql, database=DB)
query = pd.read_sql_query(
'''
SELECT
    conditions,
    sponsor_flag,
    AVG(sellerscore), MIN(sellerscore), MAX(sellerscore), STDDEV(sellerscore),
    AVG(itemprice), MIN(itemprice), MAX(itemprice), STDDEV(itemprice),
    AVG(itemsold), MIN(itemsold), MAX(itemsold), STDDEV(itemsold),
    AVG(bestoffer_flag), MIN(bestoffer_flag), MAX(bestoffer_flag), STDDEV(bestoffer_flag),
    AVG(returnsallowed_flag), MIN(returnsallowed_flag), MAX(returnsallowed_flag), STDDEV(returnsallowed_flag),
    AVG(shippingprice), MIN(shippingprice), MAX(shippingprice), STDDEV(shippingprice)
FROM
    eBay_items
GROUP BY conditions , sponsor_flag
order by conditions
''', conn)
eBay_items_summary = pd.DataFrame(query, columns=['conditions','sponsor_flag','AVG(sellerscore)','MIN(sellerscore)','MAX(sellerscore)','STDDEV(sellerscore)','AVG(itemprice)','MIN(itemprice)','MAX(itemprice)','STDDEV(itemprice)',  \
                                              'AVG(itemsold)','MIN(itemsold)','MAX(itemsold)','STDDEV(itemsold)', 'AVG(bestoffer_flag)','MIN(bestoffer_flag)','MAX(bestoffer_flag)','STDDEV(bestoffer_flag)',  \
                                              'AVG(returnsallowed_flag)','MIN(returnsallowed_flag)','MAX(returnsallowed_flag)','STDDEV(returnsallowed_flag)', 'AVG(shippingprice)','MIN(shippingprice)','MAX(shippingprice)','STDDEV(shippingprice)'])
conn.close()
eBay_items_summary

,conditions,sponsor_flag,AVG(sellerscore),MIN(sellerscore),MAX(sellerscore),STDDEV(sellerscore),AVG(itemprice),MIN(itemprice),MAX(itemprice),STDDEV(itemprice),...,MAX(bestoffer_flag),STDDEV(bestoffer_flag),AVG(returnsallowed_flag),MIN(returnsallowed_flag),MAX(returnsallowed_flag),STDDEV(returnsallowed_flag),AVG(shippingprice),MIN(shippingprice),MAX(shippingprice),STDDEV(shippingprice)
0,Manufacturer refurbished,0,135993.6863,0,2265914,269137.673888,28647.7815,4500,107900,22373.459009,...,1,0.268849,0.9281,0,1,0.258315,68.1209,0,4000,444.694236
1,Manufacturer refurbished,1,83648.5733,1039,434816,45750.261368,17929.7200,7998,59999,7213.888722,...,0,0.000000,1.0000,1,1,0.000000,0.0000,0,0,0.000000
2,New,0,6152.8860,0,115668,19258.517781,52117.5439,14999,189900,26127.820770,...,1,0.474415,0.6228,0,1,0.484684,686.7111,0,2998,937.737623
3,New,1,94150.5000,1246,187055,92904.500000,38999.5000,38999,39000,0.500000,...,1,0.500000,0.5000,0,1,0.500000,300.0000,300,300,0.000000
4,Open box,0,38393.8068,0,213419,53899.942637,23176.5909,8995,91597,14140.900734,...,1,0.317369,0.8523,0,1,0.354829,87.5965,0,1595,326.132513
5,Open box,1,46134.7500,2664,84742,38760.115695,21748.0000,12105,34997,8292.015256,...,0,0.000000,1.0000,1,1,0.000000,0.0000,0,0,0.000000
6,Seller refurbished,0,181072.5746,3,2265907,465632.861040,23847.7222,6998,99999,20996.362322,...,1,0.306518,0.9337,0,1,0.248803,111.7227,0,2400,442.108411
7,Seller refurbished,1,62518.0000,839,213416,76102.032360,14953.1613,8199,24500,4453.032827,...,1,0.176685,0.9677,0,1,0.176685,0.0000,0,0,0.000000
8,Used,0,18026.9978,0,311763,48709.153251,16344.7978,3300,102900,12085.231646,...,1,0.498937,0.5258,0,1,0.499332,436.3574,0,4995,687.276965
9,Used,1,34081.7500,29,213416,51166.194590,11700.0625,3999,38499,8192.874301,...,1,0.433013,0.8750,0,1,0.330719,135.9091,0,1000,307.754464


### Findings 

### 1) On average, the seller scores are higher for non-sponsored  as compared to sponsored items for conditions such as Manufacture Refurbished and Seller refurbished
### On the other hand sponsored items have a higher average seller score for Open Box, New and Used condition items 

### 2) For all conditions, we see that on average the price of an item is higher for non-sponsored items as compared to sponsored items ones

### 3)The average number of non-sponsored items sold is more than the average number of sponsored items sold for Manufacture Refurbished, Open Box and Used conditions
### The opposite is true for New and Seller Refurbished Items

### 4) For all conditions, we see that the average shipping price is higher for non-sponsored items as compared to sponsored items

### 5) For all conditions, apart from the New one, the proportion of items that have best offers is higher for non-sponsored than sponsored ones 

### 6) For all conditions, apart from the New one, the proportion of items  number of items that are allowed to be returned is higher for sponsored than non-sponsored ones 

### Based on the above findings we see: 
### Generally higher the item price and shipping price associated with the item, higher the chance of the item being non-sponsored
### Generally, if the item has the best offer, there is a higher chance of the item being a non-sponsored one than a sponsored one 
### Generally, if the item can be returned, there is a higher chance of the item being a sponsored one than a non-sponsored one 
### Thus the metrics such as item price, shipping price, best offer flag and returns allowed flag can be used in statistical modelling to determine if the given item is sponsored or non-sponsored.